In [49]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime

# https://www.linkedin.com/pulse/visualizing-nyc-bike-data-interactive-animated-maps-folium-toso/

lake_df = pd.read_csv('milfoil_tableB_all_years_cleaned.csv')
lake_df['corrected_longitude'] = np.where(lake_df['longitude'] > 0, lake_df['longitude']*-1, lake_df['longitude'])
lake_df['milfoil_presence'] = lake_df['milfoil_presence'].fillna('N')

In [50]:
lake_df['datetime_year'] = pd.to_datetime(lake_df['year'], format='%Y') # to_datetime requires a string
lake_df

,id,sample_point,latitude,longitude,milfoil_presence,year,corrected_longitude,datetime_year
0,0,1,42.977652,-75.887120,M,2008,-75.887120,2008-01-01
1,1,2,42.977520,-75.887213,S,2008,-75.887213,2008-01-01
2,2,3,42.977661,-75.887699,M,2008,-75.887699,2008-01-01
3,3,4,42.977674,-75.888337,M,2008,-75.888337,2008-01-01
4,4,5,42.977542,-75.889355,M,2008,-75.889355,2008-01-01
...,...,...,...,...,...,...,...,...
4555,299,300,42.960420,-75.871387,N,2022,-75.871387,2022-01-01
4556,300,301,42.961219,-75.871882,N,2022,-75.871882,2022-01-01
4557,301,302,42.961837,-75.872214,N,2022,-75.872214,2022-01-01
4558,302,303,42.948490,-75.866480,N,2022,-75.866480,2022-01-01


In [51]:
# none, trace, sparse, moderate, dense
milfoil_abundance = ['N', 'T', 'S', 'M', 'D'] # use index for number value
lake_df['milfoil_number'] = lake_df['milfoil_presence']

for index, value in enumerate(milfoil_abundance):
  lake_df['milfoil_number'] = np.where(lake_df['milfoil_presence'] == value, index, lake_df['milfoil_number'])
lake_df

,id,sample_point,latitude,longitude,milfoil_presence,year,corrected_longitude,datetime_year,milfoil_number
0,0,1,42.977652,-75.887120,M,2008,-75.887120,2008-01-01,3
1,1,2,42.977520,-75.887213,S,2008,-75.887213,2008-01-01,2
2,2,3,42.977661,-75.887699,M,2008,-75.887699,2008-01-01,3
3,3,4,42.977674,-75.888337,M,2008,-75.888337,2008-01-01,3
4,4,5,42.977542,-75.889355,M,2008,-75.889355,2008-01-01,3
...,...,...,...,...,...,...,...,...,...
4555,299,300,42.960420,-75.871387,N,2022,-75.871387,2022-01-01,0
4556,300,301,42.961219,-75.871882,N,2022,-75.871882,2022-01-01,0
4557,301,302,42.961837,-75.872214,N,2022,-75.872214,2022-01-01,0
4558,302,303,42.948490,-75.866480,N,2022,-75.866480,2022-01-01,0


In [52]:
lake_df.milfoil_number.unique()

array([3, 2, 4, 0, 1], dtype=object)

In [53]:
def create_geojson_features(df):
  features = []
  for index, row in df.iterrows():
    feature = {
      'type': 'Feature',
      'geometry': {
        'type': 'Point',
        'coordinates': [row.corrected_longitude, row.latitude]
      },
      'properties': {
        'time': str(row.datetime_year),
        'style': {'color': ''},
        'icon': 'circle',
        'iconstyle': {
          'fillColor': '#008000',
          'fillOpacity': 0.8,
          'stroke': 'true',
          'radius': df.milfoil_number # problem is here - too many values in output
        }
      }
    }
    features.append(feature)
    return(features)

start_geojson = create_geojson_features(lake_df)
start_geojson[0]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-75.88712, 42.9776517]},
 'properties': {'time': '2008-01-01 00:00:00',
  'style': {'color': ''},
  'icon': 'circle',
  'iconstyle': {'fillColor': '#008000',
   'fillOpacity': 0.8,
   'stroke': 'true',
   'radius': 0       3
   1       2
   2       3
   3       3
   4       3
          ..
   4555    0
   4556    0
   4557    0
   4558    0
   4559    0
   Name: milfoil_number, Length: 4560, dtype: object}}}

In [54]:
from folium.plugins import TimestampedGeoJson

lake_map = folium.Map(location=[42.950253, -75.870825], zoom_start=13)

TimestampedGeoJson(start_geojson, duration='P1Y', transition_time=1000, auto_play=True).add_to(lake_map)

lake_map